In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio
import h5py
import os
from six.moves import cPickle as pickle

## Load Mat
Firstly we have to load the digitStruct.mat and prepare it for the analysis.

In [2]:
train_digitStruct = h5py.File('train/digitStruct.mat')
print(train_digitStruct['digitStruct']['bbox'].shape)

(33402, 1)


The file has 33402 entries. In the next step, I will create some helper methods to read the attributes of every images like name, label, top, left, width and height.

In [3]:
#Helper Methods to read attributes of digitStruct.mat
def getSingleAttribute(struct, index, attr):
    attribute = struct[struct['digitStruct']['bbox'][index][0]][attr].value.squeeze()
    if attribute.dtype == 'float64':
        return attribute.reshape(-1)
    else:
        return np.array([struct[x].value for x in attribute]).squeeze()

def getName(struct, index):
    return struct[struct['digitStruct']['name'][index][0]].value.tostring()

def getAttributes(struct, index):
    image = getName(struct, index)
    label = getSingleAttribute(struct, index, 'label')
    for i in range(len(label)):
        if label[i] == 10:
            label[i] =0
    top = getSingleAttribute(struct, index, 'top')
    left = getSingleAttribute(struct, index, 'left')
    width = getSingleAttribute(struct, index, 'width')
    height = getSingleAttribute(struct, index, 'height')
    size = len(top)
    return image, label, top, left, width, height, size



Before I extract the features of the dataset, I need to know the max size of each attribute in order to prepare the numpy arrays.

In [4]:
label_minsize = 0

for i in range(33402):
    label_length = len(getSingleAttribute(train_digitStruct, i, 'label'))
    if label_length > label_minsize:
        label_minsize = label_length
   
print(label_minsize)

6


Through this analysis we know that we the max array size of each feature is 6.

In [ ]:
import time

def openFile(path):
    return h5py.File(path)

def initDataStructures(size):
    images = np.zeros(shape=size, dtype=str)
    labels = np.zeros(shape=(size, 6), dtype=float)
    labels.fill(10)
    tops = np.zeros(shape=(size, 6), dtype=float)
    lefts = np.zeros(shape=(size, 6), dtype=float)
    widths = np.zeros(shape=(size, 6), dtype=float)
    heights = np.zeros(shape=(size, 6), dtype=float)
    sizes = np.empty(shape=size, dtype=int)
    return images, labels, tops, lefts, widths, heights, sizes

def processing(indices, digitStruct, start=0):
    images, labels, tops, lefts, widths, heights, sizes = initDataStructures(indices)
      
    for index in range(indices):
        image, label, top, left, width, height, size = getAttributes(digitStruct, index+start)
        images[index] = image
        labels[index][:label.shape[0]] = label
        tops[index][:top.shape[0]] = top
        lefts[index][:left.shape[0]] = left
        widths[index][:width.shape[0]] = width
        heights[index][:height.shape[0]] = height
        sizes[index] = size        
        
    return images, labels, tops, lefts, widths, heights, sizes

def processData(path, validSize=0, predictSize=0, extra=False):
    digitStruct = openFile(path)
       
    if extra == True:
        validData = processing(validSize, digitStruct)
        predictData = processing(predictSize, digitStruct, start=validSize)
        return validData, predictData
    else:
        indices = digitStruct['digitStruct']['bbox'].shape[0]
        images, labels, tops, lefts, widths, heights, sizes = processing(indices, digitStruct)
        return images, labels, tops, lefts, widths, heights, sizes

In [ ]:
trainData = processData('train/digitStruct.mat')
testData = processData('test/digitStruct.mat')
validData, predictData = processData('extra/digitStruct.mat', validSize=15000, predictSize=5000, extra=True)

In [ ]:
def createFeaturesDict(dataset):
    return {
        'images': dataset[0],
        'labels': dataset[1],
        'tops': dataset[2],
        'lefts': dataset[3],
        'widths': dataset[4],
        'heights': dataset[5],
        'sizes': dataset[6]
    }

def maybePickle(train, test, valid, predict):
    pickleFile = 'svhn.pickle'
    if os.path.exists(pickleFile):
        print('The file ' + pickleFile + ' is already present.')
    else:
        print('The file ' + pickleFile + ' will be created.')
        data = {
            'train': createFeaturesDict(train), 
            'valid': createFeaturesDict(valid),
            'test': createFeaturesDict(test),
            'predict' : createFeaturesDict(predict)
        }
        try:
            with open(pickleFile, 'wb') as f:
              pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)
        except Exception as e:
            print('Unable to save data to', pickleFile, ':', e)

In [ ]:
maybePickle(trainData, testData, validData, predictData)

In [ ]:
pickleFile = open('svhn.pickle', 'r')
data = pickle.load(pickleFile)
print(data.keys())

In [ ]:
training = data['train']
testing = data['test']
valid = data['valid']


## Count of characters

In [ ]:
plt.title("Training dataset")
plt.xlabel("Count of characters")
plt.ylabel("Number of total")
plt.hist(training['sizes'], [0,1,2,3,4,5,6])
plt.show()

plt.title("Test dataset")
plt.xlabel("Count of characters")
plt.ylabel("Number of total")
plt.hist(testing['sizes'], [0,1,2,3,4,5,6])
plt.show()

plt.title("Valid dataset")
plt.xlabel("Count of characters")
plt.ylabel("Number of total")
plt.hist(valid['sizes'], [0,1,2,3,4,5,6])
plt.show()


## Attributes Exploration

In [ ]:
heightshist = np.histogram(training['heights'], bins=30)
plt.title("Distribution of height")
plt.xlabel("Number of total")
plt.ylabel("Number of heights")
x = np.array(heightshist[0])
y = np.array(heightshist[1][1:])
plt.plot(x,y)
plt.show()

topshist = np.histogram(training['tops'], bins=30)
plt.title("Distribution of top")
plt.xlabel("Number of total")
plt.ylabel("Number of top")
x = np.array(topshist[0])
y = np.array(topshist[1][1:])
plt.plot(x,y)
plt.show()

widthshist = np.histogram(training['widths'], bins=30)
plt.title("Distribution of width")
plt.xlabel("Number of total")
plt.ylabel("Number of width")
x = np.array(widthshist[0])
y = np.array(widthshist[1][1:])
plt.plot(x,y)
plt.show()

leftshist = np.histogram(training['lefts'], bins=30)
plt.title("Distribution of left")
plt.xlabel("Number of total")
plt.ylabel("Number of left")
x = np.array(leftshist[0])
y = np.array(leftshist[1][1:])
plt.plot(x,y)
plt.show()





In [ ]:
print(np.mean(heightshist[1][1:]))
print(np.mean(topshist[1][1:]))
print(np.mean(widthshist[1][1:]))
print(np.mean(leftshist[1][1:]))